In [7]:
import os
from PIL import Image
import numpy as np
# from openslide import OpenSlide

openslide_path = r'C:\Users\Andre\Documents\openslide-win64-20230414\bin'

os.environ['PATH'] = openslide_path + ";" + os.environ['PATH']
# from openslide import OpenSlide

if hasattr(os, 'add_dll_directory'):
    # Python >= 3.8 on Windows
    with os.add_dll_directory(openslide_path):
        import openslide
else:
    import openslide
    
import cv2
import numpy as np

wsisrc = r'\\169.254.138.20\Andre\data\PDAC Purity Project\PDAC PURITY Final Analysis\LCM_H&Es_10x\czi\ometiff20xv2'
tenxsrc = os.path.join(wsisrc, '5x_python')
if not os.path.exists(tenxsrc):
    os.mkdir(tenxsrc)
# wsis = [_ for _ in os.listdir(wsisrc) if _.endswith('ndpi')]
wsis = [_ for _ in os.listdir(wsisrc) if _.endswith('ndpi')]
wsis2 = [os.path.splitext(_)[0] for _ in wsis]
tenx = [_ for _ in os.listdir(tenxsrc) if _.endswith('tif')]
tenx2 = [os.path.splitext(_)[0] for _ in tenx]
missingimages = [_ for _ in wsis2 if _ not in tenx2]
for idx, missingimage in enumerate(missingimages):
    print(idx, '/', len(missingimages), ' processing: ', missingimage)
    wsi = openslide(os.path.join(wsisrc, missingimage + '.ndpi'))
    # wsi = cv2.imread(os.path.join(wsisrc, missingimage + '.tif'))
    rsf =4
    svs_img = wsi.read_region(location=(0, 0), level=0, size=wsi.level_dimensions[0]).convert('RGB')
    resize_factorx = rsf / float(
        wsi.properties['openslide.mpp-x'])  #8um = 1.25x #4um = 2.5x, #2um=5x, 1um=10x, 0.5um=20x, 0.25um=40x
    resize_factory = rsf / float(wsi.properties['openslide.mpp-y'])
    resize_dimension = tuple(
        [int(np.ceil(wsi.dimensions[0] / resize_factorx)), int(np.ceil(wsi.dimensions[1] / resize_factory))])
    svs_img10x = svs_img.resize(resize_dimension, resample=Image.NEAREST)
    svs_img10x.save(os.path.join(tenxsrc, missingimage + '.tif'), resolution=4, resolution_unit=1, quality=100,
                    compression=None)

0 / 48  processing:  PD_1 2L-1 LCM


AttributeError: 'NoneType' object has no attribute 'read_region'

In [11]:
import os
from PIL import Image  
import numpy as np

wsisrc = r'\\169.254.138.20\Andre\data\PDAC Purity Project\PDAC PURITY Final Analysis\LCM_H&Es_10x\czi\ometiff20xv2'
tenxsrc = os.path.join(wsisrc, '5x_python')
if not os.path.exists(tenxsrc):
    os.mkdir(tenxsrc)
wsis = [_ for _ in os.listdir(wsisrc) if _.endswith('tif')]
wsis2 = [os.path.splitext(_)[0] for _ in wsis]

tenx = [_ for _ in os.listdir(tenxsrc) if _.endswith('tif')]
tenx2 = [os.path.splitext(_)[0] for _ in tenx]

missingimages = [_ for _ in wsis2 if _ not in tenx2]

from PIL import Image
Image.MAX_IMAGE_PIXELS = None

for idx, missingimage in enumerate(missingimages):

    path = os.path.join(wsisrc, missingimage + '.tif')
    
    if os.path.getsize(path) < 10000:
        print(f"Skipping small file: {path}")
        continue

    print(idx, '/', len(missingimages), ' processing: ', missingimage)

    wsi = Image.open(path)
    wsi.thumbnail((1000, 1000))
    wsi = wsi.resize(wsi.size)
    
    # Handle non-RGB modes
    if wsi.mode not in ['RGB', 'RGBA']:
        wsi = wsi.convert('RGB')
        
    rsf = 4    
    resize_factor = rsf / float(wsi.info['dpi'][0]) 
    resize_dimension = tuple([int(np.ceil(dim/resize_factor)) for dim in wsi.size])
    
    wsi_resized = wsi.resize(resize_dimension, Image.NEAREST)
    wsi_resized.save(os.path.join(tenxsrc, missingimage + '.tif'))

Skipping small file: \\169.254.138.20\Andre\data\PDAC Purity Project\PDAC PURITY Final Analysis\LCM_H&Es_10x\czi\ometiff20xv2\PD_1 2L-1 LCM.tif
Skipping small file: \\169.254.138.20\Andre\data\PDAC Purity Project\PDAC PURITY Final Analysis\LCM_H&Es_10x\czi\ometiff20xv2\PD_10 3M LCM.tif
2 / 48  processing:  PD_11 2O LCM


ValueError: image has wrong mode